模組

In [ ]:
import requests
from bs4 import BeautifulSoup
# import pandas as pd
from snownlp import SnowNLP
import csv
from time import sleep
from random import uniform

頭

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'}

連結抓取

In [ ]:
def fetch_url(url):
    return requests.get(url, headers=headers)

抓取內文

In [ ]:
def parse_article_details(soup):#內文
    try:
        header = soup.find_all('span', 'article-meta-value')
        title = header[2].text
        main_container = soup.find(id='main-container')
        all_text = main_container.text.split('--')[0]
        #去頭
        contents = all_text.split('\n')[2:]
        content = '\n'.join(contents)
        s = SnowNLP(content)
        summary = s.summary(3)  # 提取三個關鍵句子
        sentiment = s.sentiments  # 情感分析
        return {'標題': title, '內容': content, '關鍵句子': summary, '情感分析': sentiment}
    except Exception as e:
        return {'錯誤': str(e)}

抓取標題，並形成字典

In [ ]:
def get_articles_from_page(page_number):
    articles_data = []
    base_url = "https://www.ptt.cc"
    url = f"{base_url}/bbs/MakeUp/index{page_number}.html"
    response = fetch_url(url)
    soup = BeautifulSoup(response.text, "html.parser")

    sections = soup.select('div.r-ent')
    keys = ["[選擇]", "[心得]", "[妝容]"]
    for section in sections:
        num = section.find('div', class_="nrec").text
        if num == '爆' or (num.isdigit() and int(num) >= 10):
            title = section.select('div.title')[0].text.strip()
            for key in keys:
                if key in title:
                    link = section.find('a')['href']
                    article_soup = fetch_url(base_url + link).text
                    article_details = parse_article_details(BeautifulSoup(article_soup, "html.parser"))
                    article_details['類別'] = key.strip('[]')
                    article_details['人氣'] = num
                    article_details['日期'] = section.find("div", class_="date").text
                    articles_data.append(article_details)
    return articles_data

main主程式（決定頁數）

In [ ]:
start_page = 3850
end_page = 3851

all_articles = []
for page in range(start_page, end_page + 1):
    all_articles.extend(get_articles_from_page(page))
    sleep(uniform(0.4,1))

輸出檔案

In [ ]:
filenames=["標題","類別","內容","人氣","日期","關鍵句子","情感分析"]
file=open("demo.csv","a+")
writor=csv.DictWriter(file,fieldnames=filenames)
writor.writerows(all_articles)